In [17]:
from google import genai
from google.genai import types
import pathlib
import httpx
import os
import json

In [18]:
# 환경 변수 불러오기
api_key = os.environ.get('MY_API_KEY')
# client 설정
client = genai.Client(api_key=api_key)

In [19]:
# 미리 정의된 스키마, 시스템 프롬포트 불러오기 (파일 이름으로 자동 탐색)
def find_file(filename, search_path='.'):
    # 주어진 파일명을 search_path부터 재귀적으로 탐색하여 경로 반환
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# 상위 디렉토리부터 탐색 시작
# 현재 노트북의 상위 디렉토리부터 탐색 시작
# base_path = pathlib.Path.cwd().parent.parent
# 현재 디렉토리부터 탐색 시작
base_path = pathlib.Path.cwd()

profile_agent_schema_path = find_file("Profile_Agent_Object.json", base_path)
profile_agent_system_prompt_path = find_file("Profile_Agent_SystemPrompt.md", base_path)

if not profile_agent_schema_path or not profile_agent_system_prompt_path:
    raise FileNotFoundError("Profile Agent 프롬포트 파일을 찾을 수 없습니다")

with open(profile_agent_schema_path, 'r', encoding='utf-8') as f:
    profile_agent_schema = json.load(f)

with open(profile_agent_system_prompt_path, 'r', encoding='utf-8') as f:
    profile_agent_system_prompt = f.read()


In [20]:
# content 생성

# 가짜 입력값
profile = {
    "learning_goal": {
        "focus_areas": ["Policy Gradient", "Clipping method"],
        "target_depth": "Deep Understanding",
        "question_modality": "Mathematical"
    },
    "user_status": {
        "proficiency_level": "Intermediate",
        "weakness_focus": True
    },
    "interaction_style": {
        "language_preference": None,
        "scenario_based": None
    },
    "feedback_preference": {
        "strictness": "Strict",
        "explanation_depth": "Detailed_with_Examples"
    },
    "scope_boundary": "Lecture_Material_Only"
}

exam_type = "5Choice"

# contents 생성시, 딕셔너리는 문자열로 바꾼다
current_session_profile_str = json.dumps(profile, ensure_ascii=False)

profile_agent_contents = [
    "Current Profile: " + current_session_profile_str,
    "Exam Type: " + exam_type
]

In [21]:
# response 생성
response = client.models.generate_content(
    model = "gemini-3-flash-preview",
    contents = profile_agent_contents,
    config={
        "system_instruction": profile_agent_system_prompt,
        "response_mime_type": "application/json",
        "response_schema": profile_agent_schema
    })

In [22]:
print(response.text)

{
  "status": "INCOMPLETE",
  "missing_info": [
    "language_preference",
    "scenario_based"
  ],
  "missing_info_queries": "문제에 사용될 용어나 설명의 주된 언어(한글/영어)를 선택해 주시고, 실무 시나리오 형태의 문제를 포함하고 싶은지 알려주세요."
}
